In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

tf.reset_default_graph()

# 먼저 학습시킬데이터와 학습목표치 데이터를 위한 리스트를 만들어보았습니다.
train_input = []
train_label = []

# a,e,i,o,u 150만개씩 있는데이터를 하나의 txt파일로 합쳐서 불러왔습니다.
# 각각의 파일의 65번쨰까지는 train, 10번째까지는 test로 나누어 325, 50명의 train, test 데이타를 구분하였습니다.
a = np.loadtxt('75Raw_a.txt',np.float32)
e = np.loadtxt('75Raw_e.txt',np.float32)
i = np.loadtxt('75Raw_i.txt',np.float32) 
o = np.loadtxt('75Raw_o.txt',np.float32) 
u = np.loadtxt('75Raw_u.txt',np.float32) 

a_train = a[0:1300000]
e_train = e[0:1300000]
i_train = i[0:1300000]
o_train = o[0:1300000]
u_train = u[0:1300000]

a_test = a[1300000:1500000]
e_test = e[1300000:1500000]
i_test = i[1300000:1500000]
o_test = o[1300000:1500000]
u_test = u[1300000:1500000]

ae = np.append(a_train,e_train)
io = np.append(i_train,o_train)
aeio = np.append(ae,io)
aeiou = np.append(aeio,u_train)
wav_all = aeiou

ae_test = np.append(a_test,e_test)
io_test = np.append(i_test,o_test)
aeio_test = np.append(ae_test,io_test)
aeiou_test = np.append(aeio_test,u_test)
test_wav_all = aeiou_test

# 불러온 데이터를 reshape하여 2000개씩 잘라내서 데이터를 구분하였습니다.
data_all = np.reshape(wav_all, [-1,20000])
x_data = data_all
test_all = np.reshape(test_wav_all,[-1,20000])

# 데이터를 필요한 만큼 자르는 부분으로 여러 방법을 써보았지만 전부 사용하는게 가장 정확한거같아 전부 사용했습니다.
x_data = x_data[:,0:20000]
test_all = test_all[:,0:20000]

(325, 20000)
(50, 20000)


In [2]:
# 학습목표치 데이터는 onehot방식 사용을 하기 위해서 0~4까지의 임의 목표치를 설정하였습니다.
# a = 0, e = 1, i = 2, o = 3, u = 4 로 정했습니다.
# while문을 사용하여 75개씩의 0,1,2,3,4 값을 가진 array를 만들었습니다.

target_value = []
test_target_value = []

while len(target_value) != 325:
    number = 0
    if len(target_value) == 65:
        number = 1
    elif len(target_value) == 130:
        number = 2
    elif len(target_value) == 195:
        number = 3
    elif len(target_value) == 260:
        number = 4
    else:
        number = 0
    for i in range(65):
        target_value.append(number)
        
while len(test_target_value) != 50:
    number = 0
    if len(target_value) == 10:
        number = 1
    elif len(target_value) == 20:
        number = 2
    elif len(target_value) == 30:
        number = 3
    elif len(target_value) == 40:
        number = 4
    else:
        number = 0
    for i in range(10):
        test_target_value.append(number)
    
# 위에서처럼 65, 10개씩의 0,1,2,3,4의 값을 가진 리스트를 다시 reshape하여 onehot방식으로 변환하였습니다.

target_value = np.reshape(target_value,[-1,1])
test_target_value = np.reshape(test_target_value,[-1,1])

y_data = target_value

enc = OneHotEncoder()
enc.fit(y_data)
y_data_onehot = enc.transform(y_data).toarray()

enc2 = OneHotEncoder()
enc2.fit(test_target_value)
test_y_data = enc.transform(test_target_value).toarray()

# 위에서처럼 만든 데이터들을 train_input, train_label, test_input, test_label로 설정하여 학습을 진행하였습니다.
# train은 훈련시키는 데이터이고 test는 테스트할때 정확도를 측정하는 데이터입니다.

train_input = x_data
train_label = y_data_onehot

test_input = test_all
test_label = test_y_data

(325, 20000)
(325, 5)
(50, 20000)
(50, 5)


C:\Users\Seo\.conda\envs\cuda\lib\site-packages\sklearn\preprocessing\_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\Seo\.conda\envs\cuda\lib\site-packages\sklearn\preprocessing\_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In c

In [3]:
# 학습을 위한 신경망구조를 짜보았습니다.

# 위에서 train_input data의 한 데이터가 20000(500,40)크기를 가지므로 아래처럼 placeholder의 크기를 지정하였습니다.

# train_label은 5개의 목표치를 위한 onehot 방식의 array이기 때문에 참고하여 Y의 placeholder의 크기를 지정하였습니다.

# 신경망은 leaky_relu방식을 사용하였으며 dropout(학습시 신경망의 일부분을 끄고 학습시키는것)을 사용하여 학습 능률을 높였습니다.

# 풀링을 할때 2*2 필터로 풀링을 하면(padding=same으로하면 크기는 유지) 원래의 크기에서 반으로 주는것을 참고하였습니다.

# 마지막 구분하는 신경망 구조를 만들때는 일반적으로 sigmoid(0~1)를 쓰지만 여기서는 목표치가 0,1,2,3,4 이기때문에 matmul을 사용했습니다.

tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None,20000])
X_wav = tf.reshape(X, [-1,500,40,1])
Y = tf.placeholder(tf.float32,[None,5])
rate = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([3, 3, 1, 16], stddev=0.01))
# Conv -> (?, 500, 40, 32)
# Pool -> (?, 250, 20, 32) 
L1 = tf.nn.conv2d(X_wav, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.leaky_relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L1 = tf.nn.dropout(L1, rate=rate)


W2 = tf.Variable(tf.random_normal([3, 3, 16, 32], stddev=0.01))
# Conv ->(?, 250, 20, 64)
# Pool ->(?, 125, 10, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.leaky_relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L2 = tf.nn.dropout(L2, rate=rate)



W3 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
# Conv ->(?, 125, 10, 64)
# Pool ->(?, 63, 5, 64)
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
L3 = tf.nn.leaky_relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L3 = tf.nn.dropout(L3, rate=rate)



W5 = tf.Variable(tf.random_normal([3, 3, 64, 32], stddev=0.01))
# Conv ->(?, 63, 5, 64)
# Pool ->(?, 32, 3, 64)
L5 = tf.nn.conv2d(L3, W5, strides=[1, 1, 1, 1], padding='SAME')
L5 = tf.nn.leaky_relu(L5)
L5 = tf.nn.max_pool(L5, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L5 = tf.nn.dropout(L5, rate=rate)

L5 = tf.reshape(L5, [-1, 32 * 32 * 3])
# reshape -> (?,64*4*4)

W6 = tf.get_variable("W6", shape=[32 * 32 * 3, 32], initializer=tf.contrib.layers.xavier_initializer())
b6 = tf.Variable(tf.random_normal([32]))
L6 = tf.nn.sigmoid(tf.matmul(L5, W6) + b6)
L6 = tf.nn.dropout(L6, rate=rate)
# sigmoid -> (?,64)

W7 = tf.get_variable("W7", shape=[32, 5], initializer=tf.contrib.layers.xavier_initializer())
b7 = tf.Variable(tf.random_normal([5]))
# shape -> (?,5)

logits = tf.matmul(L6, W7) + b7
print(np.shape(logits))

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

(?, 5)


In [17]:
# 학습을 해나가는 optimizer로 adam을 적용시켰습니다.

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)


# 변수 초기화

sess = tf.Session()
sess.run(tf.global_variables_initializer())

#batch_size는 65로 하였고 10번의 epochs로 진행했습니다.

batch_size = 65
training_epochs = 10

print("학습을 시작합니다.")

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(325 // batch_size)
 

        
    for i in range(total_batch):
        start = ((i+1) * batch_size) - batch_size
        end = ((i+1) * batch_size)
        batch_xs = train_input[start:end]
        batch_ys = train_label[start:end]
        feed_dict = {X: batch_xs, Y: batch_ys, rate : 0.3}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
        
    if epoch % 2 == 0 or epoch == training_epochs-1:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
        
        correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        print("Current accuracy:", sess.run(accuracy, feed_dict={X:test_input, Y:test_label,rate:0}))
        
        
 
print('학습이 종료되었습니다.')

# 학습을 여러번 진행해본 결과 정확도가 매우 들쑥 날쑥하는것을 볼 수 있었습니다.

# 아마 초기값 설정에 따라서 정확도가 달라진다고 생각하였고 

# 따라서 여러번 테스트를 해보아 정확도가 100%까지도 올라가기는 하는것도 볼 수 있었습니다.

학습을 시작합니다.
Epoch: 0001 cost = 3.199704167
Current accuracy: 0.0
Epoch: 0003 cost = 2.648898816
Current accuracy: 0.4
Epoch: 0005 cost = 1.786412835
Current accuracy: 0.98
Epoch: 0007 cost = 1.733755589
Current accuracy: 1.0
Epoch: 0009 cost = 1.675377822
Current accuracy: 0.98
Epoch: 0010 cost = 1.736641192
Current accuracy: 0.94
학습이 종료되었습니다.


In [18]:
# 결과는 94프로로 꽤 정확한 예측을 하는 것을 볼 수 있습니다.

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print("Accuracy:", sess.run(accuracy, feed_dict={X:test_input, Y:test_label,rate:0}))


Accuracy: 0.94
